In [1]:
import torch
import torch.nn as nn
import PIL.Image
import ivy
import numpy as np
import jax


jax.config.update("jax_enable_x64", True)


class SimpleConvNet(nn.Module):
    """
    Simple Convolutional Neural Network
    """

    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=3),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(26 * 26 * 10, 50),
            nn.ReLU(),
            nn.Linear(50, 20),
            nn.ReLU(),
            nn.Linear(20, 10),
        )
        
        self.apply(self.initialize_weights)

    def forward(self, x):
        return self.layers(x)

    def predict(self, inp):
        """predict digit for input"""
        self.eval()
        with torch.no_grad():
            raw_output = self(inp)
            _, pred = torch.max(raw_output, 1)
            return pred
        
    def initialize_weights(self, module):
        if isinstance(module, nn.Linear) or isinstance(module, nn.Conv2d):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.constant_(module.bias, 0.0)

2023-06-01 22:23:08.365131: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 22:23:08.885195: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Alpha access


In [2]:
model = SimpleConvNet()

In [4]:
img = PIL.Image.open("0.png")
tensor_img = torch.from_numpy(np_img).float()
tensor_img = tensor_img.unsqueeze(0).unsqueeze(0)
tensor_img = torch.nn.functional.interpolate(tensor_img, size=28, mode='bicubic', align_corners=False)

In [5]:
jax_graph = ivy.compiler.compiler.transpile(model, to="haiku", args=(tensor_img,))

/home/rishab/anaconda3/envs/ivy/lib/python3.10/site-packages/six.py:87: DeprecationWarning: The Tix Tk extension is unmaintained, and the tkinter.tix wrapper module is deprecated in favor of tkinter.ttk
  __import__(name)
/home/rishab/Documents/codes/unify/ivy/ivy/func_wrapper.py:234: UserWarning: Creating many views will lead to overhead when performing inplace updates with this backend
  warnings.warn(
/home/rishab/Documents/codes/unify/ivy/ivy/func_wrapper.py:234: UserWarning: Creating many views will lead to overhead when performing inplace updates with this backend
  warnings.warn(
/home/rishab/anaconda3/envs/ivy/lib/python3.10/site-packages/jax/_src/deprecations.py:51: DeprecationWarning: jax.numpy.DeviceArray is deprecated. Use jax.Array.
  warnings.warn(message, DeprecationWarning)


In [6]:
jax_graph = ivy.compiler.compiler.transpile(model, to="flax", args=(tensor_img,))

/home/rishab/Documents/codes/unify/ivy/ivy/func_wrapper.py:234: UserWarning: Creating many views will lead to overhead when performing inplace updates with this backend
  warnings.warn(


TypeError: to_flax_module.<locals>.TranspiledFlaxModule.__init__() got an unexpected keyword argument 'lazy'